In [2]:
# Utils.ipynb
import os
import torch
import logging
from tensorboardX import SummaryWriter
# tensor board를 pytorhch에서 사용할 수 있게 만든 모듈
# 머신러닝 실험에 필요한 시각화 및 도구 제공


In [ ]:

logger = logging.getLogger(__name__)
writer = SummaryWriter() # tensorboard 사용법은 추후에 추가하도록 하자!

class Metric(): # 평가지표 제작

    def __init__(self,args):
        self.args = args
    
    def get_lr(self,optimizer):
        return optimizer.state_dict()["param_groups"][0]["lr"]
        # https://tutorials.pytorch.kr/recipes/recipes/what_is_state_dict.html
        # state_dict는 모델이나 optimizer에 있는 파라미터를 확인 할 수 있게 해주는 함수.
    
    def count_parameters(self, model):
        print(sum(p.numel() for p in model.parameters() if p.requires_grad))
        # 모델의 파라미터들 중에서 파라미터가 requires_grad = True 인 경우에만
        # 행렬의 원소 갯수를 반환하고 그 총합을 출력

    def cal_acc(self,yhat,y):
        with torch.no_grad(): # auto gradient가 필요하지 않은 경우에, 불필요한 연산을 줄이고 속도 높이려고 사용
            # https://coffeedjimmy.github.io/pytorch/2019/11/05/pytorch_nograd_vs_train_eval/
            yhat = yhat.max(dim = -1)[1] 
            # tensor를 dim = 0 : batch, dim = 1 : column , dim : 2 = raw 
            # 에 대해서 max값을 구한다. dim = -1은 shape중 마지막 값에 대한 max 출력
            # 출력물을 보면, torch.max()[0]은 max값들 , torch.max()[1] 은 max값의 index 출력
            acc =  (yhat == y).float().mean()
            # accuracy를 출력하는 함수

    def cal_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))

    def cal_dev_score(self,score,indicator): 
        # score랑 indicator가 무엇을의미하는지 아직 잘 모르겠음
        # validation data : model 학습시에 사용하는 data / test 데이터와는 구분
        # validation_score는 iteration이 증가할 수록 감소하도록 설계
        # 암튼 확실한것은 validation_score는 
        validation_score = score["score"] / score["iter"]
        for key, value in indicator.items():
            # indicator가 뭔지 모르겠지만. key 값과 value를 가진다.
            indicator[key] /= score["iter"]

            print("\n\nCosine-Similarity :\tPearson: {:.4f}\tSpearman: {:.4f}".format(
                indicator['eval_pearson_cosine'], indicator['eval_spearman_cosine']))

            print("Manhattan-Distance:\tPearson: {:.4f}\tSpearman: {:.4f}".format(
                indicator['eval_pearson_manhattan'], indicator['eval_spearman_manhattan']))

            print("Euclidean-Distance:\tPearson: {:.4f}\tSpearman: {:.4f}".format(
                indicator['eval_pearson_euclidean'], indicator['eval_spearman_euclidean']))

            print("Dot-Product-Similarity:\tPearson: {:.4f}\tSpearman: {:.4f}\n".format(
                indicator['eval_pearson_dot'], indicator['eval_spearman_dot']))
    
            # dev_set을 사용해서 학습을 하면서 다양한 indicator를 사용한 결과값을 출력

            return validation_score

    def update_indicator(self, indicator, score):
        for key,value in indicator.items():
                        
            if key == 'eval_spearman_cosine':
                indicator[key] += score['eval_spearman_cosine']
            elif key == 'eval_pearson_cosine':
                indicator[key] += score['eval_pearson_cosine']
            elif key == 'eval_spearman_manhattan':
                indicator[key] += score['eval_spearman_manhattan']
            elif key == 'eval_pearson_manhattan':
                indicator[key] += score['eval_pearson_manhattan']
            elif key == 'eval_spearman_euclidean':
                indicator[key] += score['eval_spearman_euclidean']
            elif key == 'eval_pearson_euclidean':
                indicator[key] += score['eval_pearson_euclidean']
            elif key == 'eval_spearman_dot':
                indicator[key] += score['eval_spearman_dot']
            elif key == 'eval_pearson_dot':
                indicator[key] += score['eval_pearson_dot']
                
                # 사용자가 입력한 indicator에 맞게 바꿔준다.

    def draw_graph(self,cp): # tensorboard 사용법은 시간이 나면 추가하기로.
        writer.add_scalars("loss_graph",{"train":cp["t1"],"valid":cp["vl"]},cp["ep"])
        writer.add_scalars("acc_graph",{"train":cp["tma"],"valid":cp["vma"]},cp["ep"])

    def performance_check(self,cp,config):
        print(f'\t==Epoch: {cp["ep"] + 1:02} | Epoch Time: {cp["epm"]}m {cp["eps"]}s==')
        print(f'\t==Train Loss: {cp["tl"]:.4f} | Train acc: {cp["tma"]:.4f}==')
        print(f'\t==Valid Loss: {cp["vl"]:.4f} | Valid acc: {cp["vma"]:.4f}==')
        print(f'\t==Epoch latest LR: {self.get_lr(config["optimizer"]):.9f}==\n')

    def print_size_of_model(self,model):
        torch.save(model.state_dict(),"temp.p")
        # 실험 결과에서 나온 state_dict를 저장함.
        print("Size(MB) :", os.path.getsize("temp.p") / 1e6)
        # Byte 단위로 출력되는 크기를 MB 크기로 바꿔 출력 
        os.remove("temp.p")
        # 임시로 사용한 파일을 삭제한다.

    def move2device(self,sample,device):
        if len(sample) == 0:
            return {} # 이건 뭐지?
            # 받은 데이터가 0이면 비어있는 무언가를 출력한다.
        def _move_to_device(maybe_tensor, device):
            if torch.is_tensor(maybe_tensor):
                return maybe_tensor.to(device)
                    # 입력받은 값이 tensor가 맞다면 device로 
            elif isinstance(maybe_tensor,dict):
                    # 만일 tensor가 아니고 dict인 경우
                return { key: _move_to_device(value, device) for key, value in maybe_tensor.items()}
                    # maybe_tensor.items()로 key와 value를 추출하고 그중에 key에 대응하는 value들만 device로 보낸다.
            elif isinstance(maybe_tensor, list):
                    # 만일 tensor가 아니고 list인 경우
                return [_move_to_device(x, device) for x in maybe_tensor]
            elif isinstance(maybe_tensor, tuple):
                    # 만일 tensor가 아니고 list인 경우
                return [_move_to_device(x, device) for x in maybe_tensor]
            else:
                return maybe_tensor 
                    # 이도 저도 아니면, device로 보내지 않는다.
        return _move_to_device(sample, device)

    def save_model(self,config,cp,pco): # 이부분은 아직 잘 모르겠다...
            # 보통 여기서 config = configparser.ConfigParser()의 config가 입력될 것으로 생각됨.

        if not os.path.exists(config["args"].path_to_save):
            os.makedirs(config["args"].path_to_save)
                   
        sorted_path = config['args'].path_to_save + config['args'].ckpt
        if cp['vs'] > pco['best_valid_score']:
            # pco['early_stop_patient'] = 0
            pco['best_valid_score'] = cp['vs']

            state = {'model': config['model'].state_dict(),
                    'optimizer': config['optimizer'].state_dict()}

            torch.save(state, sorted_path)
            print(f'\t## SAVE {sorted_path} |'
                f' valid_score: {cp["vs"]:.4f} |'
                f' epochs: {cp["ep"]} |'
                f' steps: {cp["step"]} ##\n')
    
    def pytorch_cos_sim(a,b):

        if not isinstance(a,torch.Tensor):
            a = torch.Tensor(a)

        if not isinstance(a,torch.Tensor):
            b = torch.Tensor(b)

        if len(a.shape) == 1:
            a = a.unsqueeze(0)
            # 불필요하게 하나의 차원이 더 존재할 경우 제거한다.
        
        if len(b.shape) == 1:
            b = b.unsqueeze(0)

        a_norm = a/a.norm(dim = 1)[:,None]
        # 입력 값에서 두번째(?)에 있는 값들을 가지고 정규화 ( batch 정규회?)
        # [:,None] 하게 되면, 차윈이 하나 늘어남.
        # ex ) [3,3,3,3]짜리 tensor를 norm(dim = 1)하게 되면 size가 [3,3,3]이 되는데
        # norm(dim = 1)[:,None]을 하게 되면, 아까 사라진 자리에 차원이 하나 새로 생김.
        b_norm = b/b.norm(dim = 1)[:,None]
        return torch.mm(a_norm,b_norm.transpose(0,1))
        # 둘의 코사인 유사도 값 계산해서 출력.

In [4]:
print(torch.randn(3,3,3).norm(dim = 1))
print(torch.randn(3,3,3))

tensor([[2.8013, 1.5340, 1.9762],
        [2.6276, 1.3199, 2.8784],
        [1.4850, 0.8533, 2.4757]])
tensor([[[ 0.7300, -1.6466,  1.5072],
         [ 1.1415, -0.9067,  0.5334],
         [-2.1241,  0.7620, -0.3972]],

        [[ 0.4489,  0.3835,  0.4083],
         [ 1.3466,  0.0871, -0.3806],
         [ 1.5205, -1.6326, -1.0466]],

        [[ 2.6160, -0.9521,  0.5963],
         [-0.3956, -0.8108, -0.0930],
         [ 1.8834, -1.5518,  1.3180]]])


In [5]:
print(torch.randn(3,3,3,3).norm(dim = 1)[:,None].size())
#
print(torch.randn(3,3,3,3).norm(dim = 1).size())

torch.Size([3, 1, 3, 3])
torch.Size([3, 3, 3])


In [6]:
import torch
y = torch.rand(3,3,3)
print(y)

print(y.max(dim = 2))
print(y.max(dim = 1))

#dim = 1 : 행렬의 열 단위로 봤을 때, max값 출력
#dim = 2

tensor([[[0.3691, 0.8219, 0.6813],
         [0.0952, 0.5311, 0.3600],
         [0.6905, 0.1635, 0.5889]],

        [[0.1779, 0.0580, 0.0617],
         [0.0414, 0.6654, 0.6715],
         [0.9180, 0.7025, 0.9789]],

        [[0.3876, 0.1857, 0.7988],
         [0.1554, 0.1441, 0.1958],
         [0.3047, 0.1910, 0.4461]]])
torch.return_types.max(
values=tensor([[0.8219, 0.5311, 0.6905],
        [0.1779, 0.6715, 0.9789],
        [0.7988, 0.1958, 0.4461]]),
indices=tensor([[1, 1, 0],
        [0, 2, 2],
        [2, 2, 2]]))
torch.return_types.max(
values=tensor([[0.6905, 0.8219, 0.6813],
        [0.9180, 0.7025, 0.9789],
        [0.3876, 0.1910, 0.7988]]),
indices=tensor([[2, 0, 0],
        [2, 2, 2],
        [0, 2, 0]]))


In [7]:
import torch
isinstance(torch.tensor([3,3,3]) , tuple)

False